## Задание 2

Написать приложение и функцию, которые собирают основные новости с сайта на выбор dzen.ru, lenta.ru, mail.ru . Для парсинга использовать XPath
Структура данных должна содержать:
* название источника
* наименование новости
* ссылку на новость
* дата публикации

In [1]:
from pprint import pprint
from lxml import html
import requests
import pandas as pd

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'

In [3]:
DZEN = 'https://dzen.ru/news/'

In [4]:
headers = {
    'User-Agent': USER_AGENT,
}

params = {
    'sso_failed': '',
}

In [5]:
def get_content_dom_from_html_text(url, headers=None, params=None):
    response = requests.get(url, headers=headers, params=params)
    content_dom = html.fromstring(response.text)

    print(response.url)
    
    return content_dom

In [15]:
def parser_dzen_news(content_dom):
    news_container = content_dom.xpath("//div[contains(@class,'mg-card ')]")
    dzen_news = []
    for new_container in news_container:
        new_source = new_container.xpath(".//a[@class='mg-card__source-link']/text()")[0]
        new_text = new_container.xpath(".//h2/a/text()")[0].replace('\xa0', ' ')
        new_link = new_container.xpath(".//h2/a/@href")[0]
        new_date = new_container.xpath(".//span[@class='mg-card-source__time']/text()")[0]
        news_dict = {
            'new_source': new_source,
            'new_text': new_text,
            'new_link': new_link,
            'new_date': new_date,
        }
        dzen_news.append(news_dict) 

    return dzen_news


In [20]:
def get_news(content_dom, parser_name='dzen'):
    result = None
    
    if parser_name == 'dzen':
      result = parser_dzen_news(content_dom)
    
    return result

In [21]:
dom = get_content_dom_from_html_text(DZEN, headers=headers, params=params)

https://dzen.ru/news?issue_tld=ru&sso_failed=


In [22]:
dzen_news = get_news(dom, 'dzen')
dzen_news

[{'new_source': 'РИА Новости',
  'new_text': 'Минобороны: российские военные пресекли попытку массированной атаки дронов на Крым',
  'new_link': 'https://dzen.ru/news/story/Minoborony_rossijskie_voennye_presekli_popytku_massirovannoj_ataki_dronov_naKrym--8f9e44d55ac21c105602ff962c709e28?lang=ru&rubric=index&fan=1&stid=oZ5dE1ZACLUXybBs3ZlC&t=1677674603&tt=true&persistent_id=2128160525&story=db558f48-3da5-5def-884f-60a48fa8647a&issue_tld=ru',
  'new_date': '22:37'},
 {'new_source': 'ТАСС',
  'new_text': 'Пригожин сообщил, что десятки тысяч бойцов ВСУ оказывают сопротивление в Артемовске',
  'new_link': 'https://dzen.ru/news/story/Prigozhin_soobshhil_chto_desyatki_tysyach_bojcov_VSU_okazyvayut_soprotivlenie_vArtemovske--9a723af51150fd9eab6abbb1064f123c?lang=ru&rubric=index&fan=1&stid=SJDxe0iViWT40sKc8akw&t=1677674603&tt=true&persistent_id=2202960001&story=22055d10-a2fb-53bb-8f9c-37ffb49ff73c&issue_tld=ru',
  'new_date': '22:34'},
 {'new_source': 'ТАСС',
  'new_text': 'В Москве начался наб

In [23]:
pd.DataFrame(dzen_news)

,new_source,new_text,new_link,new_date
0,РИА Новости,Минобороны: российские военные пресекли попытк...,https://dzen.ru/news/story/Minoborony_rossijsk...,22:37
1,ТАСС,"Пригожин сообщил, что десятки тысяч бойцов ВСУ...",https://dzen.ru/news/story/Prigozhin_soobshhil...,22:34
2,ТАСС,В Москве начался набор волонтеров ко Дню Побед...,https://dzen.ru/news/story/VMoskve_nachalsya_n...,22:36
3,Газета.Ru,Песков напомнил о поддержке Киевом националист...,https://dzen.ru/news/story/Peskov_napomnil_opo...,22:22
4,РИА Новости,Кириенко: обучение демобилизованных бойцов СВО...,https://dzen.ru/news/story/Kirienko_obuchenie_...,22:39
...,...,...,...,...
60,Коммерсантъ,РБК: автодилер «Авилон» приобретет завод Volks...,https://dzen.ru/news/story/RBK_avtodiler_Avilo...,22:35
61,Автостат,АВТОВАЗ с 1 марта 2023 года повысит цены на ав...,https://dzen.ru/news/story/AVTOVAZ_s1_marta_20...,22:01
62,Motor,Концерн Volkswagen запретил китайским заводам ...,https://dzen.ru/news/story/Koncern_Volkswagen_...,11:02
63,Автоновости дня,В России начались продажи новых минивэнов Hyun...,https://dzen.ru/news/story/VRossii_nachalis_pr...,21:47
